In [29]:
import codecs
import nltk.data
import pymorphy2
import re
import os
import nltk
import pandas as pd
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
from itertools import takewhile
morph = pymorphy2.MorphAnalyzer()

In [60]:
delims = ",.!?/&:;@'*()«»...\n\t\`"
stopwords = open("stopwords.txt", 'r').read().split("\n")

# Delete delimiters, remove stopwords, make float . stay as _
def process(text):
    title = text.split()[0]
    text = re.sub(r'\[.+?\]\s*', ' ', text)
    text = re.sub(r'«.+?»\s*', ' ', text)
    text = re.sub(r'-', ' ', text)
    text = re.sub(r' — ', ' . ', text)
    text = re.sub(r'([+-]?[0-9]+)\.([0-9]+)',r'\1_\2', text)
    text = re.sub(r'([+-]?[0-9]+),([0-9]+)',r'\1_\2', text)
    lines = [w.strip() for w in re.split("["+"\\".join(delims)+"]", text.lower()) if w != ""]
    return title, [' '.join(filter(lambda x: x.lower() not in stopwords and re.match(r'^-?[А-Яа-я]+$', x), text.split())) for text in lines]
 
# if there is n-gram keyword with high certanity, delete related n-1-grams
def remove_dups(less, more, num):
    words = [x[0] for x in less]
    todel = []
    for s in more:
        l = s[0].split(' ')
        cnt = 0
        for i in range(len(l) - num + 1):
            if not " ".join(l[i: i + num]).strip() in words:
                break
            cnt = cnt + 1
        if cnt == 2:
            todel += [" ".join(l[i: i + num]) for i in range(len(l) - num + 1)]
    return list(filter(lambda s: s[0] not in todel, less))

# check if difference is only in wordforms
def are_alike(s_a, s_b):
    a = s_a.split(' ')
    b = s_b.split(' ')
    for i in range(len(a)):
        if morph.parse(a[i])[0].normal_form != morph.parse(b[i])[0].normal_form:
            return False
    return True

# remove keywords that differ in wordform only
def remove_wordforms(l):
    new_l = []
    for a in range(len(l)):
        flag = True
        for b in range(a):
            if a != b:
                if are_alike(l[a], l[b]):
                    flag = False
                    break
        if flag:
            new_l.append(l[a])
    return new_l

def get_confidence(ngram_freq, nless1_freq):
    return (ngram_freq * ngram_freq * ngram_freq) / float(nless1_freq) if ngram_freq > 1 and nless1_freq > 1 else 0

In [55]:
#text = [process(open("wiki/" + f, 'r').read())
#        for f in os.listdir("wiki")
#        if f.endswith(".txt")]

In [56]:
text = open("train_content.csv", 'r').read()

In [57]:
t = process(text)

In [58]:
def get_keywords(data, n1 = 5, n2 = 15, n3 = 10, n4 = 5, n5 = 5, b1 = 0.01):
    total = 0.0
    monograms = []
    bigrams = []
    trigrams = []
    fourgrams = []
    fivegrams = []
    # get ngrams
    for line in data:
        if len(line) == 0:
            continue
        line_len = len(line.split(" "))
        total += line_len

        token = nltk.word_tokenize(line)
        monograms += ngrams(token, 1)
        if line_len < 2:
            continue
        bigrams += ngrams(token, 2)
        if line_len < 3:
            continue
        trigrams += ngrams(token, 3)
        if line_len < 4:
            continue
        fourgrams += ngrams(token, 4)
        if line_len < 5:
            continue
        fivegrams += ngrams(token, 5)

    # get frequencies
    d1 = Counter(monograms)
    d2 = Counter(bigrams)
    d3 = Counter(trigrams)
    d4 = Counter(fourgrams)
    d5 = Counter(fivegrams)

    # get probabilities of occurence of ngram in text, choose keywords based on probability and frequence
    key1 = sorted([(" ".join(x), float(d1[x] / total)) 
            for x in d1], key=lambda tup: tup[1], reverse=True)[:n1]
    # if float(d1[x] / total) > b1]
    key2 = sorted([(" ".join(x), get_confidence(d2[x], d1[(x[0],)])) 
            for x in d2], key=lambda tup: tup[1], reverse=True)[:n2]
    key1 = remove_dups(key1, key2, 1)
    
    key3 = sorted([(" ".join(x), get_confidence(d3[x], d2[(x[0], x[1],)])) 
            for x in d3], key=lambda tup: tup[1], reverse=True)[:n3]
    key2 = remove_dups(key2, key3, 2)

    key4 = sorted([(" ".join(x), get_confidence(d4[x], d3[(x[0], x[1], x[2],)])) 
            for x in d4], key=lambda tup: tup[1], reverse=True)[:n4]
    key3 = remove_dups(key3, key4, 3)    
    
    key5 = sorted([(" ".join(x),get_confidence(d5[x], d4[(x[0], x[1], x[2], x[3],)])) 
            for x in d5], key=lambda tup: tup[1], reverse=True)[:n5]
    key4 = remove_dups(key4, key5, 4)
    key1 = remove_wordforms([x[0] for x in key1])
    key2 = remove_wordforms([x[0] for x in key2])
    key3 = remove_wordforms([x[0] for x in key3])
    key4 = remove_wordforms([x[0] for x in key4])
    key5 = remove_wordforms([x[0] for x in key5])
    keywords = key1 + key2 + key3 + key4 + key5
    print(keywords)

In [61]:
get_keywords(t[1], n1 = 20, n2 = 20, n3 = 20, n4 = 5, n5 = 1, b1 = 0.1)

['жизнь', 'день', 'любовь', 'человек', 'женщина', 'хочу', 'нужно', 'сердце', 'мужчина', 'друг', 'глаза', 'тобой', 'хорошо', 'жить', 'друг друга', 'люблю тебя', 'новый год', 'посланник аллаха', 'следующий день', 'благословит аллах', 'участники группы', 'данный момент', 'благословение аллаха', 'моей жизни', 'класс статус', 'пишите личку', 'оптовым ценам', 'первую очередь', 'подробности читайте здесь', 'благословит аллах приветствует', 'мир благословение аллаха', 'товары оптовым ценам', 'знать людей читают', 'ставьте важно знать', 'прочитали ставьте важно', 'уважаемые участники группы', 'стоимость тура человека', 'между мужчиной женщиной', 'поставьте класс статус', 'важно знать людей', 'наступающим новым годом', 'клуб совместных покупок тольятти', 'покупок тольятти самара жигулевск', 'класс статус класс статус', 'тольятти самара жигулевск вход', 'статус класс статус класс', 'совместных покупок тольятти самара жигулевск']


In [6]:
with open('out.txt','w') as f:
    for (title, data) in text:    
        f.write(title + "\n")
        print(title)
        get_keywords(data)
        for string in keywords:
            f.write(string + ", ")
        f.write("\n\n")
        print("\n\n")

Android 
['android', 'google', 'операционной системы', 'исходный код', 'ос android', 'material design', 'серверы компании', 'вынес решение', 'intellij idea', 'open handset alliance', 'версия операционной системы', '23 сентября 2008', 'идентификационной информации серверы', 'ноября опубликован список', 'android native development kit', 'android 4_0 ice cream sandwich', 'передачи идентификационной информации серверы компании']



API
['api', 'функции', 'операционных систем', 'семантика функции', 'типов аргументов', 'синтаксической конструкции', 'низкоуровневых компонентов', 'объявления функции', 'низкого уровня', 'включает описание', 'области видимости', 'графических интерфейсов', 'часть directx', 'системные вызовы ос', 'api графических интерфейсов', 'api операционных систем', 'используют api низкоуровневых компонентов']



Java Virtual Machine
['java', 'jvm', 'sun', 'microsoft', 'байт', 'virtual machine', 'байт кода', 'скачивания сайте', 'спецификации jvm', 'runtime environment', 'фирмы